In [1]:
%load_ext cython

In [2]:
import numpy as np
from scipy.misc import electrocardiogram

In [3]:
m = 4
r = 0.5
x_ = x[0, :, 0]
N = x_.size
B = 0
A = 0

xmi = np.array([x_[i : i + m] for i in range(N - m)])
xmj = np.array([x_[i : i + m] for i in range(N - m + 1)])

NameError: name 'x' is not defined

In [36]:
np.sum([np.sum(np.max(np.abs(xmii - xmj), axis=1) <= r) - 1 for xmii in xmj[:-1]])

81514

In [3]:
def sampen(L, m, r, v=False):
    N = len(L)
    B = 0.0
    A = 0.0

    # Split time series and save all templates of length m
    xmi = np.array([L[i : i + m] for i in range(N - m)])
    xmj = np.array([L[i : i + m] for i in range(N - m + 1)])

    # Save all matches minus the self-match, compute B
    B = np.sum([np.sum(np.abs(xmii - xmj).max(axis=1) <= r) - 1 for xmii in xmi])

    # Similar for computing A
    m += 1
    xm = np.array([L[i : i + m] for i in range(N - m + 1)])

    A = np.sum([np.sum(np.abs(xmi - xm).max(axis=1) <= r) - 1 for xmi in xm])

    # Return SampEn
    if v: print(A, B)
    return -np.log(A / B)

def sampen1(L, m, r, v=False):
    N = len(L)
    B = 0
    A = 0
    
    for i in range(N-m):
        X = L[i:i+m]
        for ii in range(N-m+1):
            if np.max(np.abs(L[ii:ii+m] - X)) <= r:
                B += 1
    
    for i in range(N-m):
        X = L[i:i+m+1]
        for ii in range(N-m):
            if np.max(np.abs(L[ii:ii+m+1] - X)) <= r:
                A += 1
    A -= N-m
    B -= N-m
    if v: print(A, B)
    return -np.log(A/B)

In [5]:
%%cython
# cython: infer_types = True
# cython: boundscheck = False
# cython: wraparound = False
import numpy as np
cimport cython
from libc.math cimport log, exp, ceil, sqrt, isnan, pow, abs

cdef void mean_sd_1d(const double[:] x, double* mean, double* std):
    cdef Py_ssize_t n = x.size, i

    cdef double k = x[0]
    cdef double Ex = 0., Ex2 = 0.
    mean[0] = 0.

    for i in range(n):
        mean[0] += x[i]
        Ex += x[i] - k
        Ex2 += (x[i] - k)**2
    
    std[0] = sqrt((Ex2 - (Ex**2 / n)) / (n - 1))
    mean[0] /= n

    
def c_sampen1(const double[:] L, int m, double r):
    cdef Py_ssize_t N = L.size
    cdef int A = 0, B = 0
    
    cdef double[::1] X = np.zeros(m+1)
    
    for i in range(N-m):
        X[:m] = L[i:i+m]
        for ii in range(N-m+1):
            if np.max(np.abs(np.subtract(L[ii:ii+m], X[:m]))) <= r:
                B += 1
    
    for i in range(N-m):
        X[:] = L[i:i+m+1]
        for ii in range(N-m):
            if np.max(np.abs(np.subtract(L[ii:ii+m+1], X))) <= r:
                A += 1
    A -= N-m
    B -= N-m
    return -log(A/B)

def se(const double[:] x, int L, double r, bint v):
    cdef Py_ssize_t n = x.size
    
    cdef int A = 0, B = 0
    
    cdef int[:] run = np.zeros(n, dtype=np.intc), lastrun = np.zeros(n, dtype=np.intc)
    
    for j in range(n-1):
        xj = x[j]
        for jj in range(j+1, n):
            if abs(x[jj] - xj) <= r:
                run[jj-j-1] = lastrun[jj-j-1] + 1
                if run[jj-j-1] >= L:
                    A += 1
                if run[jj-j-1] >= L+1:
                    B += 1
            else:
                run[jj-j-1] = 0
        for k in range(n-j-1):
            lastrun[k] = run[k]
    
    if v: print(A, B)
    
    return -log(A/B)
    
    
def sample_entropy_3d(const double[:, :, :] signal, int M, double r, bint v):
    cdef Py_ssize_t n = signal.shape[1], k = signal.shape[2], p = signal.shape[0]
    entropy = np.zeros((p, M, k))
    cdef double[:, :, :] ent = entropy
    cdef long nj, j
    cdef long[:] run = np.zeros(n, dtype=int), lastrun = np.zeros(n, dtype=int)
    cdef long N, M1
    cdef double[:, :, :] A = np.zeros((p, M, k)), B = np.zeros((p, M, k))
    cdef double Y1

    cdef Py_ssize_t i, jj, mm, kk, axis, wind
    for wind in range(p):
        for axis in range(k):
            run[:] = 0
            lastrun[:] = 0
            for i in range(n-1):
                nj = n - i - 1
                Y1 = signal[wind, i, axis]
                for jj in range(nj):
                    j = jj + i + 1
                    if (((signal[wind, j, axis] - Y1) < r) and ((Y1 - signal[wind, j, axis]) < r)):
                        run[jj] = lastrun[jj] + 1
                        M1 = M if (M < run[jj]) else run[jj]
                        for mm in range(M1):
                            A[wind, mm, axis] += 1
                            if (j < (n - 1)):
                                B[wind, mm, axis] += 1
                    else:
                        run[jj] = 0
                for kk in range(nj):
                    lastrun[kk] = run[kk]

    N = <long>(n * (n - 1) / 2)

    for wind in range(p):
        for axis in range(k):
            ent[wind, 0, axis] = -log(A[wind, 0, axis] / N)

            for mm in range(1, M):
                ent[wind, mm, axis] = -log(A[wind, mm, axis] / B[wind, mm - 1, axis])
    if v: print(np.array(A), np.array(B))
    return entropy

In [6]:
x = electrocardiogram()
print(x.shape)

(108000,)


In [12]:
print(sampen(x[:10000].flatten(), 4, 0.5, True))
# print(sampen1(x.flatten(), 4, 0.5, True))
# print(c_sampen1(x.flatten(), 4, 0.5))

48904480 50349924
0.029128101903168637


In [13]:
sample_entropy_3d(x[:10000].reshape((1, -1, 1)), 4, 0.5, True)

[[[27756290.]
  [26766596.]
  [25835983.]
  [24996454.]]] [[[27749192.]
  [26759617.]
  [25829107.]
  [24989672.]]]


array([[[0.58846052],
        [0.03605203],
        [0.03512571],
        [0.03276806]]])

In [65]:
%timeit sampen(x[0, :, 0], 4, 0.5)

39.7 ms ± 1.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [68]:
%timeit sample_entropy_3d(x, 4, 0.5, False)

1.79 ms ± 46 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
